### Combinatorial Purged Cross Validation

In this section, we introduce a new method that addresses the main drawbacks of walk-forward and cross-validation techniques. Professor Prado calls this method Combinatorial Purged Cross-Validation (CPCV). Given the number of backtest paths $\varphi$ that the researcher aims for, CPCV generates the exact combinations of test/training sets needed to create these paths, while simultaneously removing observations with leaked information.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [5]:
from quant_free.finml.labeling.labeling import *

triple_barrier_event = pd.read_parquet('./research/Data/AAPL_triple_barrier_events.parquet')
avg_uniqueness = pd.read_parquet('./research/Data/AAPL_avg_unique.parquet')
feature_matrix = pd.read_parquet('./research/Data/AAPL_feature_matrix.parquet')

labels = meta_labeling(
    triple_barrier_event, 
    feature_matrix['Close']
)
triple_barrier_event['side'] = labels['bin']
meta_labels = meta_labeling(
    triple_barrier_event, # with side labels
    feature_matrix['Close']
)

#### 1. 조합적 분할

Let's consider the case of partitioning $ T $ observations into $ N $ groups without shuffling. In this scenario, the size of groups $ n = 1, \dots, N-1 $ is $\left[ \frac{T}{N} \right]$, while the size of the $ N $-th group is $ T - \left[ \frac{T}{N} \right] (N-1) $, where $[.]$ denotes the floor or integer function. 

For a test set of size $ k $, the number of possible training/test splits is given by:

$$
{N \choose N-k} = \frac{\prod_{i=0}^{k-1} \left( N-i\right)}{k!}
$$

Each combination contains $ k $ tested groups, so the total number of tested groups is $ k {N \choose N-k} $. Since all combinations are calculated, these groups are uniformly distributed across all $ N $. This implies that we can perform a total of $ \varphi \left[ N, k\right] $ backtests for a test set of size $ k $ from $ N $ groups:

$$
\varphi \left[ N, k\right] = \frac{k}{N} {N \choose N-k} = \frac{\prod_{i=0}^{k-1} \left( N-i\right)}{(k-1)!}
$$

The figure below illustrates the configuration of training/test splits for $ N = 6 $ and $ k=2 $. There are ${6 \choose 4} = 15$ splits, indexed as $ S1, \dots, S15 $. The groups marked with an "x" belong to the test set, while those without markings belong to the training set. Each group contributes to forming $ \varphi[6, 2] = 5 $ test sets, thus the training/test splitting method generates 5 backtest paths.

![1](images/1.png) 

The figure below illustrates the allocation of a single backtest path to each tested group. For example, Path 1 combines predictions from groups $ (G1, S1), (G2, S1), (G3, S2), (G4, S3), (G5, S4), (G6, S5) $, while Path 2 combines predictions from $ (G1, S2), (G2, S6), (G3, S6), (G4, S7), (G5, S8), (G6, S9) $.


![2](images/2.png) 


This represents a path generated by training the classifier on the data portion defined by $ \theta = \frac{1-k}{N} $. Although theoretically, learning is possible when $ \theta < \frac{1}{2} $, in practice, it is assumed that $ k \leq \frac{K}{2} $. The data portion of the training set, $ \theta $, increases as $ N \rightarrow T $ but decreases as $ k \rightarrow \frac{N}{2} $. The number of paths $ \varphi \left[ N, k\right] $ increases with $ N \rightarrow T $ and $ k \rightarrow \frac{N}{2} $. In the limit, the maximum number of paths can be achieved by setting $ N = T $ and $ k = \frac{N}{2} = \frac{T}{2} $, at the cost of using only half of the data for training in each combination.

In [8]:
feature_matrix['side'] = triple_barrier_event['side'].copy()
feature_matrix['label'] = meta_labels['bin'].copy()
feature_matrix.drop(['Open','High','Low','Close','Adj Close','Volume'], axis = 1, inplace = True)
feature_matrix.dropna(inplace = True)
matrix = feature_matrix[feature_matrix['side'] != 0]

X = matrix.drop(['side','label'], axis = 1)
y = matrix['label']

X_train, X_test = X.loc[:'2019'], X.loc['2020':]
y_train, y_test = y.loc[:'2019'], y.loc['2020':]

In [16]:
samples_info_sets = triple_barrier_event.loc[X_train.index].loc[:'2019', 't1']

In [17]:
from quant_free.finml.cross_validation.combinatorial import CombinatorialPurgedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

comb_purge_fold = CombinatorialPurgedKFold(
    n_splits = 5, 
    n_test_splits = 2, 
    samples_info_sets = samples_info_sets, 
    pct_embargo = 0.01
)

for train_indices, test_indices in comb_purge_fold.split(X_train, y_train):
    X_train_valid, X_test_valid = X_train.iloc[train_indices], X_train.iloc[test_indices]
    y_train_valid, y_test_valid = y_train.iloc[train_indices], y_train.iloc[test_indices]

    clf = RandomForestClassifier(random_state = 42)
    clf.fit(X_train_valid, y_train_valid)
    
    y_pred = clf.predict(X_test_valid)
    accuracy = accuracy_score(y_test_valid, y_pred)
    print(f'Accuracy: {accuracy:.4f}')

Accuracy: 0.8866
Accuracy: 0.8786
Accuracy: 0.8618
Accuracy: 0.8769
Accuracy: 0.9042
Accuracy: 0.9010
Accuracy: 0.9201
Accuracy: 0.9002
Accuracy: 0.9129
Accuracy: 0.9041
